In [2]:
# https://nbconvert.readthedocs.io/en/latest/removing_cells.html
# use these magic spells to update your classes methods on-the-fly as you edit them:
%reload_ext autoreload
%autoreload 2
from pprint import pprint
from IPython.core.display import display, HTML, Markdown
import ipywidgets as widgets
# %run includeme.ipynb # include a notebook from this same directory
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_19880/1348678174.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, Markdown


In [3]:
# common libs
import math, os, io, json, cv2, random, logging
import numpy as np
# images
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
homie=os.path.expanduser("~")
print("your home path is", homie)
fodir=os.path.join(homie,'fiftyone')
print("fiftyone dowloads data to", fodir)
try:
    os.mkdir(fodir)
except FileExistsError:
    pass

your home path is /home/sampsa
fiftyone dowloads data to /home/sampsa/fiftyone


In [5]:
# fiftyone
import fiftyone as fo
import fiftyone.zoo as foz

In [6]:
# CompressAI-Vision
from compressai_vision.conversion import nokiaBSToOpenImageV6, imageIdFileList

We expect that you have downloaded correct images and segmentation masks into open-images-v6 folder:

In [7]:
dir_=os.path.join(fodir,"open-images-v6")
print("contents of", dir_,":")
!tree --filelimit=10 $dir_ | cat

contents of /home/sampsa/fiftyone/open-images-v6 :
/home/sampsa/fiftyone/open-images-v6
├── info.json
└── validation
    ├── data [8189 entries exceeds filelimit, not opening dir]
    ├── labels
    │   ├── classifications.csv
    │   ├── detections.csv
    │   ├── masks [16 entries exceeds filelimit, not opening dir]
    │   ├── relationships.csv
    │   └── segmentations.csv
    └── metadata
        ├── attributes.csv
        ├── classes.csv
        ├── hierarchy.json
        ├── image_ids.csv
        └── segmentation_classes.csv

5 directories, 10 files


So the downloaded images reside in ``~/fiftyone/open-images-v6/data`` and segmentation masks in ``~/fiftyone/open-images-v6/labels/masks``.

We are not going to use the default OpenImageV6 annotations: MPEG VCM working group provides us with custom-format annotation files we need to convert into OpenImageV6 format.  For the detector ground truths, these are:
```
detection_validation_5k_bbox.csv           detection bbox annotations
detection_validation_labels_5k.csv         image-level annotations
detection_validation_input_5k.lst          list of images used
```

In [8]:
## TODO: DEFINE YOUR PARTICULAR PATHS
path_to_nokia_files="/home/sampsa/silo/interdigital/siloai-playground/sampsa/nokia/data5K"
path_to_images=os.path.join(fodir,"open-images-v6/validation/data")

list_file=os.path.join(path_to_nokia_files, "detection_validation_input_5k.lst")
bbox_csv_file=os.path.join(path_to_nokia_files, "detection_validation_5k_bbox.csv")
validation_csv_file=os.path.join(path_to_nokia_files, "detection_validation_labels_5k.csv")

assert(os.path.exists(bbox_csv_file)), "can't find bbox file"
assert(os.path.exists(validation_csv_file)), "can't find labels file"
assert(os.path.exists(path_to_images)), "can't find image directory"

Now we convert nokia proprietary format annotation into proper OpenImageV6 format dataset and place it into ``~/fiftyone/nokia-detection``

First, remove any previously imported stuff:

In [9]:
!rm -rf ~/fiftyone/nokia-detection

In [10]:
nokiaBSToOpenImageV6(
    validation_csv_file=validation_csv_file,
    list_file=list_file,
    bbox_csv_file=bbox_csv_file,
    output_directory=os.path.join(fodir,"nokia-detection"),
    data_dir=path_to_images
)

let's see what we got:

In [11]:
!tree --filelimit=10 ~/fiftyone/nokia-detection | cat

/home/sampsa/fiftyone/nokia-detection
├── data -> /home/sampsa/fiftyone/open-images-v6/validation/data
├── labels
│   ├── classifications.csv
│   └── detections.csv
└── metadata
    ├── attributes.csv
    ├── classes.csv
    └── image_ids.csv

3 directories, 5 files


We have a new OpenImageV6 formatted data/directory structure with new annotations, but it uses images from the official OpenImageV6 dataset (note that link from ``data -> ~/fiftyone/open-images-v6/validation/data``)

The only thing we're left to do, is to register this OpenImageV6 formatted dataset into fiftyone:

In [12]:
# remove the dataset in the case it was already registered in fiftyone
try:
    fo.delete_dataset("nokia-detection")
except ValueError as e:
    print("could not delete because of", e)

In [14]:
dataset_type = fo.types.OpenImagesV6Dataset
dataset_dir = os.path.join(fodir,"nokia-detection")
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
    label_types=("detections","classifications"),
    load_hierarchy=False,
    name="nokia-detection",
    image_ids=imageIdFileList(list_file)
)

 100% |███████████████| 5000/5000 [16.9s elapsed, 0s remaining, 290.3 samples/s]      


In [17]:
dataset.persistent=True # without this, your dabatase will disappear!

In [18]:
## now, in the future, just do
dataset = fo.load_dataset("nokia-detection")

Finaly, let's also create a dummy dataset for debugging and testing with only one sample:

In [ ]:
try:
    fo.delete_dataset("nokia-detection-dummy")
except ValueError:
    print("no dummmy dataset yet..")
dummy_dataset=fo.Dataset("nokia-detection-dummy")
for sample in dataset[0:1]:
    dummy_dataset.add_sample(sample)
dummy_dataset.persistent=True